###### The file has been successfully uploaded. The next step is to open the cluster on which you will run the notebook. Then, navigate to the cluster configurations, select “Libraries,” and click on “Install New” as shown below.Select “Maven” as the Library source.<br/> In the **Coordinates** field, copy and paste the following: **com.crealytics:spark-excel_2.12:0.13.5**. Alternatively, you can choose the latest version by clicking on “Search Packages,” and then proceed to click “Install.”

In [0]:

from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
.appName("ReadExcelWithHeader") \
.config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
.getOrCreate()

# Define the path to your Excel file
#excel_file_path = "/FileStore/tables/aircraft_data_updated.xlsx"
#excel_file_path = "/FileStore/tables/aircraft_data_updated___Copy.xlsx"
#excel_file_path = "/FileStore/tables/*.xlsx"

df = spark.read \
.format("com.crealytics.spark.excel") \
.option("header", "true") \
.option("inferSchema", "true") \
.load(excel_file_path)

# Show the DataFrame
display(df)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-4262721296679852>, line 18
      9 # Define the path to your Excel file
     10 #excel_file_path = "/FileStore/tables/aircraft_data_updated.xlsx"
     11 #excel_file_path = "/FileStore/tables/aircraft_data_updated___Copy.xlsx"
     12 excel_file_path = "/FileStore/tables/*.xlsx"
     14 df = spark.read \
     15 .format("com.crealytics.spark.excel") \
     16 .option("header", "true") \
     17 .option("inferSchema", "true") \
---> 18 .load(excel_file_path)
     20 # Show the DataFrame
     21 display(df)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - star

In [0]:
.option("header", "true") \ #use first row as header
.option("inferSchema", "true") \ #to infer schema from file
.option("dataAddress", "'SheetName'!A3:E393") \ #to select specific cells or sheet(give only sheet name) only
.option("treatEmptyValuesAsNulls", "true") \ #to treat empty values as null
.option("timestampFormat", "yyyy-MM-dd HH:mm:ss") \ #specify timestamp format
.option("dateFormat", "yyyy-MM-dd") \ #specify date format
.option("workbookPassword", "your_password") \ #to specify password for protetced files

In [0]:
import os
import glob



In [0]:
# Compine excel files to CSV file.

from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("ReadExcelWithHeader") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()

# Define the directory containing Excel files
excel_dir_path = "/FileStore/tables"

# List all files in the directory using dbutils.fs.ls
all_files = dbutils.fs.ls(excel_dir_path)

# Filter to get only the .xlsx files
excel_files = [file.path for file in all_files if file.path.endswith(".xlsx")]

# Initialize an empty DataFrame
df_combined = None

# Loop through each Excel file and read it into a DataFrame
for excel_file in excel_files:
    df = spark.read \
        .format("com.crealytics.spark.excel") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(excel_file)
    
    # Combine the DataFrames
    if df_combined is None:
        df_combined = df
    else:
        df_combined = df_combined.union(df)

# Check if df_combined is not None before writing to CSV
if df_combined is not None:
    # Define the output CSV file path
    csv_file_path = "/FileStore/tables/output_file.csv"
    
    # Save the combined DataFrame as a CSV file
    df_combined.write.csv(csv_file_path, header=True, mode='overwrite')
    
    print(f"Excel files in {excel_dir_path} have been successfully converted to {csv_file_path}")
else:
    print(f"No Excel files found in {excel_dir_path}")

# Stop the Spark session
#spark.stop()


Excel files in /FileStore/tables have been successfully converted to /FileStore/tables/output_file.csv


In [0]:
https://learn.microsoft.com/en-us/azure/databricks/dev-tools/databricks-utils

In [0]:
%fs rm -r "/FileStore/tables/output_file.csv"

res0: Boolean = true

In [0]:
# Define the pattern to search for
FileNamePattern = "Q*"
ExtentionPattern = ".xlsx"

# List all files in the directory
all_files = dbutils.fs.ls("dbfs:/FileStore/tables/salesleads/raw/")

# Filter files that match the pattern
#case of set 1 varibale without *
#filtered_files = [file.path for file in all_files if file.name.startswith(pattern.split('*')[0]) and file.name.endswith(pattern.split('*')[-1])]

#case of set 2 varibale without *
#filtered_files = [file.path for file in all_files if file.name.startswith(FileNamePattern) and file.name.endswith(ExtentionPattern)]

#case of set 2 varibale with *
filtered_files = [file.path for file in all_files if not file.name.startswith(FileNamePattern.split('*')[0]) and file.name.endswith(ExtentionPattern.split('.')[-1])]

# Print the filtered file paths
for file in filtered_files:
    print(file)
